# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from dotenv import load_dotenv
import panel as pn
from panel.interact import interact
from panel import widgets

%matplotlib inline

In [3]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Import Data

In [4]:
# Import the CSVs to Pandas DataFrames
file_path_sfo = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path_sfo, index_col="year")

file_path_geo = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path_geo)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [5]:
# Define Panel Visualization Functions
def get_housing_units_per_year():
    """Housing Units Per Year."""

    average_housing_units = sfo_data.groupby('year').mean()
    average_housing_units = average_housing_units[['housing_units']]
    housing_units_per_year = average_housing_units.plot.bar(title="Average Housing Units in San Francisco 2010-2016", legend=None)
    plt.ylim(370000,387500)
    plt.ylabel('housing units')


    return housing_units_per_year

def get_average_gross_rent_plot():
    """Average Gross Rent in San Francisco Per Year."""

    average_gross_rent = sfo_data.groupby('year').mean()
    average_gross_rent = average_gross_rent[['gross_rent']]
    average_gross_rent
    average_gross_rent.plot.line(title="Average Gross Rent in San Francisco 2010-2016", legend=None)
    plt.ylabel('gross rent ($)')

    return average_gross_rent



def get_average_sale_price_plot():
    """Average Sales Price Per Year."""
    
    average_sale_price = sfo_data.groupby('year').mean()
    average_sale_price = average_sale_price[['sale_price_sqr_foot']]

    average_sale_price_plot = average_sale_price.plot.line(title="Average Sale Price in San Francisco 2010-2016", legend=None)
    plt.ylabel('average sale price ($)')

    return average_sale_price_plot



def get_average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    sfo_average_data = sfo_data.groupby(['year','neighborhood']).mean()
    average_price_by_neighborhood = sfo_average_data.hvplot.line(groupby='neighborhood',x='year', y='sale_price_sqr_foot', label="Average Sale Price", widget_location='top')

    return get_average_price_by_neighborhood



def get_top_ten_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    sfo_average_data = sfo_data.groupby(['neighborhood']).mean()
    sfo_average_data.iloc[:,0]
    sfo_average_data_sorted = sfo_average_data.iloc[:,0].sort_values(ascending=False)
    sfo_top_ten_sale_price = sfo_average_data_sorted.head(10)
    top_ten_most_expensive_neighborhoods = sfo_top_ten_sale_price.hvplot.bar(ylabel='sale price ($)',x='neighborhood', ylim=(600,950),y='sale_price_sqr_foot', rot=60,label="Top 10 Sale Price")

    return top_ten_most_expensive_neighborhoods



def get_parallel_coordinates():
    """Parallel Coordinates Plot."""

    average_sale_price = sfo_data.groupby('neighborhood').mean()
    average_sale_price.reset_index(inplace=True)
    average_sale_price.sort_values("sale_price_sqr_foot", inplace=True, ascending=False)
    parallel_coordinates = px.parallel_coordinates(average_sale_price, color =
    'sale_price_sqr_foot', color_continuous_scale=px.colors.sequential.Inferno,
    labels={"sale_price_sqr_foot": "Sale Price",
        "housing_units": "Housing Units",
        "gross_rent": "Gross Rent"})

    return parallel_coordinates
       


def get_parallel_categories():
    """Parallel Categories Plot."""
    average_sale_price = sfo_data.groupby('neighborhood').mean()
    average_sale_price.reset_index(inplace=True)
    average_sale_price.sort_values("sale_price_sqr_foot", inplace=True, ascending=False)
    parallel_categories = px.parallel_categories(
    average_sale_price,
    dimensions=["neighborhood","sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
        "sale_price_sqr_foot": "Sale Price",
        "housing_units": "Housing Units",
        "gross_rent": "Gross Rent",
    },
)
    return parallel_categories



def get_neighborhood_map():
    """Neighborhood Map"""
    
    average_sale_price = sfo_data.groupby('neighborhood').mean()
    average_sale_price.reset_index(inplace=True)
    average_sale_price.sort_values("sale_price_sqr_foot", inplace=True, ascending=False)
    average_sale_price_geo_data = pd.concat([average_sale_price,        df_neighborhood_locations], axis='columns', join='inner')
    average_sale_price_geo_data.drop(['Neighborhood'], axis=1, inplace=True)
    load_dotenv()
    map_box_api = os.getenv("MAPBOX_API_KEY")
    px.set_mapbox_access_token(map_box_api)
    neighborhood_map = px.scatter_mapbox(
    average_sale_price_geo_data,
    lat="Lat",
    lon="Lon",
    size= "sale_price_sqr_foot",
    color= "sale_price_sqr_foot", 
    zoom=4,title="Average Sale Price in San Francisco 2010-2016",
    labels={"sale_price_sqr_foot": "Sale Price (sq/ft)"})

    return neighborhood_map


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
bar_column = pn.Column("## San Francisco Average Housing Data",get_housing_units_per_year(), get_average_sale_price_plot(), get_average_gross_rent_plot(), get_average_price_by_neighborhood(), get_top_ten_most_expensive_neighborhoods())

statistics_column = pn.Column("## Statistics Data", get_parallel_coordinates(), get_parallel_categories())

geo_column = pn.Column("## Geo Data", get_neighborhood_map())

In [ ]:
sfo_housing_dashbord = pn.Tabs(
    ("Housing Data", bar_column),
    ("Statistical Data", statistics_column),
    ("Geo Housing Data", geo_column))

## Serve the Panel Dashboard

In [ ]:
sfo_housing_dashbord.servable()